# 🦜🔗 Langchain Demo

Hallo und herzlich Willkommen!


## Überblick

### OpenAI-Modelle

ChatGPT oder auch jedes andere LLM benutzen ist relativ einfach mit Langchain

In diesen Test nutzen wir das "gpt-4-turbo" Model - mögliche Large Language Modelle von OpenAI sind:

- `gpt-35-turbo` das günstigste und am historisch am weitesten verbreitete Modell
- `gpt-4` das neue und bessere GPT Modell
- `gpt-4-turbo` Turbo-Variante von gpt-4 (günstiger, schneller, kleinere maximale Länge des Text-Outputs)
- `gpt-4-vision` ein "multimodales" Modell, welches auch auf Bilder trainiert wurde.
- `gpt-4o` die neueste multimodale Variante von gpt-4. In manchen Aspekten besser, in manchen schlechter als die alten Modelle.

OpenAI trainiert diese Versionen laufend neu, was dazu führen kann, dass Anfragen an das LLM plötzlich andere Antworten geben.
Möchte man dies verhindern, kann man seine Applikation auf einen Snapshot (z. B. gpt-4-0613) festsetzen.
Dies ist insbesondere wichtig, wenn die Applikation vom Output des LLM bestimmte Strukturen erwartet, etwa eine bestimmte XML-Syntax o.Ä.

OpenAI-Modelle werden nicht nur von OpenAI selbst gehostet, sondern auch von Azure.
Diese muss man auf dem Azure Portal selbst als Endpunkte konfigurieren, in der Regel leiden die OpenAI Azure Deployments weniger unter hoher Auslastung

### Andere Modelle

Auch wenn wir nicht damit arbeiten werden, ist es vielleicht relativ gut, die Namen der "großen" Konkurrenz-Modelle einmal gehört zu haben:

- `Gemini` das neueste Google-Modell. Es hat den Fokus vornehmlich auf multimodalem Input.
- `Claude` Claude ist die LLM-Reihe von Anthropic. Enorme Kontextlänge, oft beeindruckende Ergebnisse, teuer.
- `Mistral` ein kleines, offenes Modell von Mistral AI, auf fast jeder Hardware selbst betreibbar.
- `Mistral large` das kommerzielle Angebot von Mistral
- `Mixtral` ein hervorragendes Open-Source Modell von Mistral AI. Ein guter Kandidat für ein selbst gehostetes LLM.
- `LLama 3` das aktuelle Modell von Meta, das in einigen Bereichen an die Performance von GPT-4 heranreicht.

### DSGVO - konform?

Gerücht aus den Anfangstagen von AI. Inzwischen sind alle großen Modelle DSGVO-konform betreibbar.

### Aleph Alpha

Stand 2024 Anfang: Das aktuelle Aleph Alpha ist ein "last generation" Sprachmodell, das auch zu kleinen modernen Modellen wie Mistral 7B nicht mehr konkurrenzfähig ist.

### Temperatur

Alle LLMs sind nicht deterministisch. Aber die Temperatur ist ein Parameter, mit der man die Variabilität von Antworten hoch und runterschrauben kann.
Wie bei normalen Atomen ist die Bewegung niedrig, wenn die Temperatur niedrig ist. Wenn man die Temperatur hochschraubt, wird viel gewackelt.
Der Temperatur-Parameter ist üblicherweise ein Fließkommawert zwischen 0 und 1.

### Bitte etwas schneller

Recht neu auf dem Markt mit beeindruckendem Token/Sekunde-Verhältnis: [Groq](https://groq.com/)

### Links:

- https://python.langchain.com/docs/get_started/introduction
- https://platform.openai.com/docs/models/


#### Wir probieren aus:


In [6]:
from helpers import llm

print(
    llm()
    .invoke(
        "Hi OpenAI! Kannst du mir einen Trinkspruch auf Würzburg im fränkischen Dialekt sagen?"
    )
    .content
)

Klar, hier ist ein Trinkspruch auf Fränkisch für Würzburg:

„Auf’s Wohl, ihr Leut’, in Würzburg, da schäumt das Bier,  
Mir stoß’n an und feiern, das Leben ist hier!  
Mit Herz und mit Freundschaft, so trink’n mir vereint,  
In unserm schönen Würzburg, wo’s immer am besten scheint!“

Prost!


#### Jetzt nochmal mit Streaming. Dazu rufen wir nicht invoke sondern astream auf (a für async). Wir drehen etwas an der Temperatur, damit die Ergebnisse spannend bleiben


In [7]:
chunks = []
async for chunk in llm(temperature=1).astream(
    "Erkläre in einem Satz Quantenmechanik für 4jährige, ohne dabei Details auszulassen."
):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

Quantenmechanik ist wie ein magisches Spiel, in dem winzige Teilchen manchmal wie kleine Kügelchen und manchmal wie Wellen tanzen und sich verstecken können, und das alles auf eine Weise, die wir mit unseren Augen nicht so einfach sehen können.

## Token

Token sind die kleinste Einheit des LLM. Das haben wir gerade beim Streaming schön gesehen. Der Stream kommt Token für Token aus dem LLM gepurzelt.

Das LLM rechnet aus der Eingabe und den bisher errechneten Token die Wahrscheinlichkeit für den nächsten Token aus. Dieser neue Token wird dann angehängt und der nächste Token wird ermittelt.

So geht das immer weiter. Bis der nächste wahrscheinlichste Token ein Stop-Zeichen ist. Auf diese Weise generieren LLMs die wahrscheinlichste Fortführung der Eingabetoken.

Token können Wörter, machmal sogar Wortgruppen oder auch nur einzelne oder mehrere Buchstaben sein.

Die Bepreisung der LLMs ist an die Tokenanzahl (Eingabe und Ausgabe) gekoppelt.

Links:

- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb


In [8]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4-0125-preview")

tokens = encoding.encode("AI ist eine tolle Sache.")
print(tokens)

decoded_tokens = [
    encoding.decode_single_token_bytes(token).decode("utf-8") for token in tokens
]
for token in decoded_tokens:
    print(token)

[15836, 6127, 10021, 39674, 273, 328, 1815, 13]
AI
 ist
 eine
 tol
le
 S
ache
.


## ✅ Aufgabe

Aus welchen Token besteht der String "Berlin"?


In [ ]:
print()

## Prompt Engineering und Templates in Langchain

Um die Dinge von der AI zu bekommen, die man erwartet, stellt man am besten sehr konkrete und präzise Anfragen.

Weil eine AI oft an ein bestimmtes Feld von Aufgaben gekoppelt ist, gibt man die Rahmenanweisung dann in ein Template ein, um nicht immer wieder die gleiche Rahmenanweisung zu schreiben.

Die jeweilige konkrete Nutzeranfrage wird dann in das Template eingefügt und das ausgefüllte Template ans LLM übergeben.

Der Trend geht immer mehr zu Chat-Modellen. Hierbei ist die Information, die man dem LLM gibt, in "Messages" unterteilt. Besondere Gewichtung hat eine System-Message. Diese kann Rahmenanweisungen enthalten, an die sich das LLM halten soll. Dem Nutzer wird es schwer fallen, das LLM dazu zu bewegen, sich über eine Anweisung in der System-Message hinweg zu setzen. Das LLM wurde ganz einfach darauf trainiert, sich an die Anweisungen einer System-Message strikt zu halten.

### Links

- https://python.langchain.com/docs/get_started/quickstart#prompt-templates
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering
- https://learnprompting.org/docs/intro
- https://www.promptingguide.ai/
- https://smith.langchain.com/hub


In [9]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist eine hilfsbereiter {beruf} aus Würzburg."),
        (
            "human",
            "Erkläre in 2 Sätzen im lokalen Dialekt warum Deine Kunden aus {ort} die besten sind.",
        ),
    ]
)

print(prompt.format(beruf="Entwickler", ort="Würzburg"))

System: Du bist eine hilfsbereiter Entwickler aus Würzburg.
Human: Erkläre in 2 Sätzen im lokalen Dialekt warum Deine Kunden aus Würzburg die besten sind.


### Langchain Hub Beispiel

Weil das "Prompt-Engineering" ein bisschen Übung braucht und es diverse Tricks gibt, hat LangChain einen "Hub", auf dem man eine ganze Reihe vorgefertigter Prompts für verschiedene Anwendungsfälle findet.

Dort kann man sich inspirieren lassen, Prompts forken oder auch selbst etwas für andere Leute zur Verfügung stellen, wenn es sich als nützlich erweist.

Links:

- https://smith.langchain.com/hub/borislove/customer-sentiment-analysis


In [10]:
from langchain import hub

sentiment_prompt = hub.pull("borislove/customer-sentiment-analysis")

client_letter = """Ich bin von dem Volleyballschläger zutiefst enttäuscht. Zuerst ist der Griff abgefallen, danach auch noch der Dynamo. Außerdem riecht er noch schlechter als er schmeckt. Wieso ist das immer so ein Ärger mit euch?"""
format_instructions = """Zusätzlich zur numerischen Klassifizierung sollst du herausfinden, was der Kunde gerne gehabt hätte. Antworte auf deutsch."""

print(
    sentiment_prompt.format(
        client_letter=client_letter, format_instructions=format_instructions
    )
)

System: As a customer service representative, you receive the following email from a customer.
Your task is to identify the customer's sentiment and categorize it based on the scale below:
    0 - Calm: Customer asks questions but does not seem upset; is just seeking information.
    1 - Slightly Frustrated: Customer shows subtle signs of irritation but is still open to solutions.
    2 - Frustrated: Customer explicitly states being unhappy or irritated but is willing to discuss a solution.
    3 - Very Frustrated: Customer is clearly agitated, uses strong language, or mentions the problem repeatedly.
    4 - Extremely Frustrated: Customer is intensely unhappy, may raise their voice or use aggressive language.
    5 - Overwhelmed: Customer seems emotionally upset, says things like 'I can't take this anymore' or 'This is the worst experience ever.'
If you cannot identify the sentiment for some reason, simply respond with 'Unknown'
Human: Letter: 
'''Ich bin von dem Volleyballschläger zu

/Users/lorenz/code/langchain_agents/.venv/lib/python3.11/site-packages/langsmith/client.py:5402: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


## Jetzt fängt es an, etwas technischer zu werden. Wieso heißt LangChain eigentlich LangChain?

Langchain definiert einige Python-Operatoren neu, wenn sie zwischen LangChain-Objekten stehen. Der bekannteste ist die Pipe: |

Wenn die Pipe zwischen zwei Langchain-Objekten steht, wird die Ausgabe des ersten Obekts an das nächste weitergegeben. Damit erhält man eine "Chain" von "Runnables"

#### Links

- https://python.langchain.com/docs/modules/chains


In [12]:
from langchain.schema import StrOutputParser  # Hilft beim Formatieren
from helpers import llm

chain = prompt | llm() | StrOutputParser()
print(chain.invoke({"beruf": "Winzer", "ort": "Würzburg]"}))

Die Kunden aus Würzburg sind einfach klasse, weil sie unser Handwerk schätzen und immer für ein gutes Glas Wein zu haben sind. Außerdem gibt's hier in der Region so viel schöne Tradition und Geselligkeit, da macht's doppelt Spaß, mit ihnen zusammenzuarbeiten!


In [13]:
# Streaming
async for chunk in chain.astream({"beruf": "Winzer", "ort": "Würzburg"}):
    print(chunk, end="", flush=True)

Die Leut' aus Würzburg sin einfach die besten, weil sie unsere Weine mit so viel Herz und Leidenschaft schätzen! Außerdem kenn' ich hier jeder und jede, und des macht's einfach besonders, wenn mer zusammen an Glas Wein genießt.

In [14]:
# Funktioniert auch das Beispiel vom Hub?
sentiment_chain = sentiment_prompt | llm() | StrOutputParser()
async for chunk in sentiment_chain.astream(
    {"client_letter": client_letter, "format_instructions": format_instructions}
):
    print(chunk, end="", flush=True)

Der Kunde zeigt deutlich Unzufriedenheit und Frustration über das Produkt, insbesondere über die Qualität des Volleyballschlägers, da mehrere Teile defekt sind und er eine unangenehme Geruchswahrnehmung beschreibt. 

Basierend auf der Skala würde ich die Stimmung des Kunden als 2 - Frustrated einstufen, da er ausdrücklich erwähnt, dass er enttäuscht ist und die Situation als ärgerlich empfindet, aber dennoch offen für eine Diskussion über Lösungen zu sein scheint.

Der Kunde hätte sich wahrscheinlich gewünscht, dass der Volleyballschläger von besserer Qualität ist und dass solche Probleme wie ein abfallender Griff und ein defekter Dynamo nicht vorkommen. 

Wenn Sie weitere Unterstützung benötigen, lassen Sie es mich bitte wissen!

In [21]:
# Wir können dynamisch die format_instructions des Templates überschreiben, um neue Ergebnisse zu bekommen
sentiment_chain = sentiment_prompt | llm(model="gpt-4o-mini") | StrOutputParser()
format_instructions = """Zusätlich zur sentiment Analysis ist es deine Aufgabe, die Sinnhaftigkeit der Kundenäußerung zu überprüfen."""
async for chunk in sentiment_chain.astream(
    {"client_letter": client_letter, "format_instructions": format_instructions}
):
    print(chunk, end="", flush=True)

Sentiment: 3 - Very Frustrated

Die Kundenäußerung ist sinnvoll, da der Kunde konkrete Probleme mit dem Produkt beschreibt (Griff und Dynamo sind abgefallen, unangenehmer Geruch und Geschmack) und seine Enttäuschung über die Erfahrung mit dem Unternehmen ausdrückt.

## ✅ Aufgabe

An Stelle der Frage nach Sinnhaftigkeit soll eine Vorschlag gemacht werden, was man dem Kunden antworten kann.


In [ ]:
format_instructions = """[CHANGE HERE]"""
async for chunk in sentiment_chain.astream(
    {"client_letter": client_letter, "format_instructions": format_instructions}
):
    print(chunk, end="", flush=True)

### Debug Informationen gewünscht?


In [22]:
from langchain.globals import set_debug

In [23]:
# Und jetzt selber mal Ausprobieren
set_debug(True)
print(chain.invoke({"beruf": "Programmierer", "ort": "[INSERT]"}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "beruf": "Programmierer",
  "ort": "[INSERT]"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "beruf": "Programmierer",
  "ort": "[INSERT]"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Du bist eine hilfsbereiter Programmierer aus Würzburg.\nHuman: Erkläre in 2 Sätzen im lokalen Dialekt warum Deine Kunden aus [INSERT] die besten sind."
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Meinere Kunden aus Würzburg san die besten, weil se immer kreativ und offen für neue Ideen san. Außerdem schätze ich die herzliche Art und die Verbindlichkeit von de Leut hier – das macht's einfach besonders!",